# Notebook 4: Entanglement

📘 **Notebook 4/7**: Entanglement  
⏱️  **Estimated Duration**: 15 min  
🎯 **What You'll Learn**:
   • Distinguish classical vs quantum correlations
   • Measure qubits in different bases
   • Test Bell's inequality (CHSH)
   • Prove quantum non-locality

**Progress**: ⬛⬛⬛⬛⬛⬜⬜ (4/7)

---

## 🔮 When Einstein Was Wrong

In Notebook 3, we created **Bell states** - qubits with perfect correlations.

**But**: Can't we reproduce these correlations classically? 🤔

**Spoiler**: NO! And we're about to prove it mathematically! 🎉

## Section 1: Environment Setup

Let's import our quantum tools and set up for entanglement experiments.

In [ ]:
# Quantum computing framework
from qiskit import QuantumCircuit, transpile
from qiskit.quantum_info import Statevector
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_bloch_multivector, plot_state_qsphere

# Visualization and numerical tools
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

# Custom plotting utilities
import sys
sys.path.append('..')
from utils.plotting import configure_beautiful_plots, COLORS

# Configure beautiful plots
configure_beautiful_plots()

# Create simulator
simulator = AerSimulator()

print("✅ All imports successful!")
print("✅ Ready to explore quantum entanglement")

## 🤔 INTUITION: Classical Correlations

Before diving into quantum, let's see how correlations work classically.

## Section 2: Classical Correlations - Shared Secret

**Scenario**: Alice and Bob share a classical correlation.

**Example**: Before separating, they flip a coin together:
- If HEADS → Alice gets "0", Bob gets "0"
- If TAILS → Alice gets "1", Bob gets "1"

Then they separate (arbitrarily far apart).

**When they measure**:
- Alice opens her envelope → sees 0 or 1
- Bob opens his envelope → sees the **same** value

**Result**: Perfect correlation! 100% identical results.

### 🎭 Key Point

The values were **predetermined** before measurement.  
Each person had a **hidden variable** (their envelope).

This is called a **local hidden variable theory**.

In [ ]:
# Classical simulation: shared secret
import random

def classical_correlation_experiment(n_trials=1000):
    """Simulate classical perfect correlation with hidden variable"""
    alice_results = []
    bob_results = []
    
    for _ in range(n_trials):
        # Shared hidden variable (predetermined)
        hidden_value = random.randint(0, 1)
        
        # Both get the same predetermined value
        alice_results.append(hidden_value)
        bob_results.append(hidden_value)
    
    return alice_results, bob_results

# Run experiment
alice, bob = classical_correlation_experiment(1000)

# Count agreements
agreements = sum(1 for a, b in zip(alice, bob) if a == b)
print(f"📊 Classical Correlation Results (1000 trials):")
print(f"   • Agreements: {agreements}/1000 = {agreements/10:.1f}%")
print(f"   • Disagreements: {1000-agreements}/1000 = {(1000-agreements)/10:.1f}%")

# Visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5), dpi=150)

# Alice's results
alice_counts = Counter(alice)
ax1.bar([str(k) for k in alice_counts.keys()], alice_counts.values(),
        color=COLORS['primary'], edgecolor='black', linewidth=2, alpha=0.8)
ax1.set_title("Alice's Measurements", fontsize=13, fontweight='bold')
ax1.set_ylabel('Counts', fontsize=11)
ax1.set_xlabel('Result', fontsize=11)
ax1.grid(axis='y', alpha=0.3)

# Bob's results  
bob_counts = Counter(bob)
ax2.bar([str(k) for k in bob_counts.keys()], bob_counts.values(),
        color=COLORS['secondary'], edgecolor='black', linewidth=2, alpha=0.8)
ax2.set_title("Bob's Measurements", fontsize=13, fontweight='bold')
ax2.set_ylabel('Counts', fontsize=11)
ax2.set_xlabel('Result', fontsize=11)
ax2.grid(axis='y', alpha=0.3)

plt.suptitle('Classical Correlation: 100% Agreement', fontsize=15, fontweight='bold')
plt.tight_layout()
plt.show()

print("\n✅ Classical correlation: perfect agreement!")
print("   💡 Explanation: The value was predetermined (hidden variable)")

## 💡 CONCEPT: Quantum Correlations

Now let's create quantum correlations with a Bell state and see if they behave the same way.

## Section 3: Quantum Correlations - Bell State

**Recall from Notebook 3**: Bell state |Φ+> = (|00> + |11>)/√2

**Circuit**:
```
q0 (Alice): ──H──●──
                 │
q1 (Bob):   ─────X──
```

**Properties**:
- Before measurement: qubits in **superposition** of |00> and |11>
- **No predetermined values!** (unlike classical)
- After measurement: 50% chance of both getting 0, 50% chance of both getting 1
- Results are **always identical**

### 🎯 Crucial Question

**Is this just like the classical case?**

With a single measurement basis (Z basis), **we can't tell the difference!**

Both give 100% correlation. 🤔

**But**: What if we measure in **different bases**? 🔬

In [ ]:
# Create Bell state and measure in Z basis
qc_bell = QuantumCircuit(2, 2)
qc_bell.h(0)        # Hadamard on Alice's qubit
qc_bell.cx(0, 1)    # CNOT to entangle
qc_bell.measure([0, 1], [0, 1])

print("Circuit for Bell state |Φ+>:")
display(qc_bell.draw('mpl'))

# Execute
counts = simulator.run(qc_bell, shots=1000).result().get_counts()

print(f"\n📊 Quantum Correlation Results (1000 shots):")
for state, count in sorted(counts.items()):
    print(f"   |{state}>: {count} times ({count/10:.1f}%)")

# Count agreements (both 0 or both 1)
agreements = counts.get('00', 0) + counts.get('11', 0)
disagreements = counts.get('01', 0) + counts.get('10', 0)
print(f"\n   • Agreements (00 or 11): {agreements}/1000 = {agreements/10:.1f}%")
print(f"   • Disagreements (01 or 10): {disagreements}/1000 = {disagreements/10:.1f}%")

# Visualization
fig, ax = plt.subplots(figsize=(10, 6), dpi=150)
bars = ax.bar(counts.keys(), counts.values(),
              color=COLORS['quantum'], edgecolor='black', linewidth=2, alpha=0.8)

ax.set_xlabel('Measured state (Alice, Bob)', fontsize=12)
ax.set_ylabel('Number of times', fontsize=12)
ax.set_title('Quantum Correlation in Z Basis: |Φ+> = (|00> + |11>)/√2', 
             fontsize=14, fontweight='bold')
ax.grid(axis='y', alpha=0.3)

# Annotate
ax.text(0.5, 0.95, 'Perfect correlation: 100% identical results!',
        transform=ax.transAxes, ha='center', va='top', fontsize=12,
        bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.5))

plt.show()

print("\n✅ In Z basis: quantum and classical look identical!")
print("   Both show 100% correlation...")
print("\n🤔 But are they really the same? Let's test another basis!")

## 🎯 CHECKPOINT 1: Can you...

Before continuing, verify your understanding:

- [ ] Explain classical correlation with hidden variables?
- [ ] Create a Bell state with H + CNOT?
- [ ] Understand that in Z basis, both give 100% correlation?

If yes to all, let's discover measurement in different bases! 🚀

---

## 💡 CONCEPT: Measuring in Different Bases

So far, we've only measured in the **Z basis** (computational basis |0>, |1>).

But qubits can be measured in **any basis**!

## Section 4: The X Basis - Hadamard Before Measurement

### 📐 What is the X Basis?

Instead of measuring |0> vs |1>, we can measure **|+> vs |->**:

- **|+>** = (|0> + |1>)/√2 → "horizontal" on Bloch sphere
- **|->** = (|0> - |1>)/√2 → "horizontal opposite"

### 🔄 How to Measure in X Basis?

**Trick**: Apply **Hadamard before measurement**!

**Why?** Because:
- H transforms |+> → |0>
- H transforms |-> → |1>

So measuring in Z basis after H = measuring in X basis before H!

### 🎯 Circuit

```
q: ──[Gates]──H──M──
                  ↓
            X-basis measurement
```

In [ ]:
# Demonstrate X basis measurement
# Test 1: Measure |+> in X basis → should give 0 (100%)
qc_test1 = QuantumCircuit(1, 1)
qc_test1.h(0)       # Create |+>
qc_test1.h(0)       # H before measurement (X basis)
qc_test1.measure(0, 0)

counts1 = simulator.run(qc_test1, shots=1000).result().get_counts()

# Test 2: Measure |-> in X basis → should give 1 (100%)
qc_test2 = QuantumCircuit(1, 1)
qc_test2.x(0)       # Create |1>
qc_test2.h(0)       # Create |-> = H|1>
qc_test2.h(0)       # H before measurement (X basis)
qc_test2.measure(0, 0)

counts2 = simulator.run(qc_test2, shots=1000).result().get_counts()

print("🧪 X Basis Measurement Tests:")
print(f"\nTest 1 - Measure |+> in X basis:")
print(f"   Result: {counts1}")
print(f"   ✅ Always 0 (|+> eigenstate of X)")

print(f"\nTest 2 - Measure |-> in X basis:")
print(f"   Result: {counts2}")
print(f"   ✅ Always 1 (|-> eigenstate of X with eigenvalue -1)")

# Visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5), dpi=150)

ax1.bar(counts1.keys(), counts1.values(),
        color=COLORS['primary'], edgecolor='black', linewidth=2, alpha=0.8)
ax1.set_title('Measuring |+> in X Basis', fontsize=13, fontweight='bold')
ax1.set_ylabel('Counts', fontsize=11)
ax1.set_xlabel('Result', fontsize=11)
ax1.grid(axis='y', alpha=0.3)

ax2.bar(counts2.keys(), counts2.values(),
        color=COLORS['secondary'], edgecolor='black', linewidth=2, alpha=0.8)
ax2.set_title('Measuring |-> in X Basis', fontsize=13, fontweight='bold')
ax2.set_ylabel('Counts', fontsize=11)
ax2.set_xlabel('Result', fontsize=11)
ax2.grid(axis='y', alpha=0.3)

plt.suptitle('X Basis Measurements', fontsize=15, fontweight='bold')
plt.tight_layout()
plt.show()

print("\n💡 Key insight: We can measure in ANY basis by applying the right gate before measurement!")

## Section 5: Bell State in X Basis

**Now the interesting part**: What happens if we measure our Bell state |Φ+> in the X basis?

### 🧮 Mathematical Analysis

Bell state: |Φ+> = (|00> + |11>)/√2

In X basis, we can rewrite this using:
- |0> = (|+> + |->)/√2
- |1> = (|+> - |->)/√2

After substitution and simplification:
$$|\Phi^+\rangle = \frac{1}{\sqrt{2}}(|++\rangle + |--\rangle)$$

**Prediction**: 
- 50% chance both measure |+> (result: 00)
- 50% chance both measure |-> (result: 11)
- 0% chance of measuring |+-> or |-+> (results: 01 or 10)

**Still 100% correlation in X basis!** 🤯

In [ ]:
# Measure Bell state in X basis
qc_bell_x = QuantumCircuit(2, 2)

# Create Bell state
qc_bell_x.h(0)      # Alice: create superposition
qc_bell_x.cx(0, 1)  # CNOT to entangle

# Measure in X basis (apply H before measurement)
qc_bell_x.h(0)      # Alice measures in X basis
qc_bell_x.h(1)      # Bob measures in X basis
qc_bell_x.measure([0, 1], [0, 1])

print("Circuit for Bell state measured in X basis:")
display(qc_bell_x.draw('mpl'))

# Execute
counts_x = simulator.run(qc_bell_x, shots=1000).result().get_counts()

print(f"\n📊 Bell State in X Basis (1000 shots):")
for state, count in sorted(counts_x.items()):
    interpretation = "|++>" if state == "00" else "|-->" if state == "11" else f"|{state}>"
    print(f"   {state} → {interpretation}: {count} times ({count/10:.1f}%)")

# Count agreements
agreements_x = counts_x.get('00', 0) + counts_x.get('11', 0)
disagreements_x = counts_x.get('01', 0) + counts_x.get('10', 0)
print(f"\n   • Agreements (++ or --): {agreements_x}/1000 = {agreements_x/10:.1f}%")
print(f"   • Disagreements (+- or -+): {disagreements_x}/1000 = {disagreements_x/10:.1f}%")

# Visualization - Compare Z basis and X basis
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6), dpi=150)

# Z basis results (from before)
qc_bell_z = QuantumCircuit(2, 2)
qc_bell_z.h(0)
qc_bell_z.cx(0, 1)
qc_bell_z.measure([0, 1], [0, 1])
counts_z = simulator.run(qc_bell_z, shots=1000).result().get_counts()

ax1.bar(counts_z.keys(), counts_z.values(),
        color=COLORS['primary'], edgecolor='black', linewidth=2, alpha=0.8)
ax1.set_title('Bell State in Z Basis', fontsize=13, fontweight='bold')
ax1.set_ylabel('Counts', fontsize=11)
ax1.set_xlabel('Result (00 = |00>, 11 = |11>)', fontsize=10)
ax1.grid(axis='y', alpha=0.3)
ax1.text(0.5, 0.9, '100% correlation!',
         transform=ax1.transAxes, ha='center', fontsize=11,
         bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.5))

ax2.bar(counts_x.keys(), counts_x.values(),
        color=COLORS['secondary'], edgecolor='black', linewidth=2, alpha=0.8)
ax2.set_title('Bell State in X Basis', fontsize=13, fontweight='bold')
ax2.set_ylabel('Counts', fontsize=11)
ax2.set_xlabel('Result (00 = |++>, 11 = |-->)', fontsize=10)
ax2.grid(axis='y', alpha=0.3)
ax2.text(0.5, 0.9, '100% correlation!',
         transform=ax2.transAxes, ha='center', fontsize=11,
         bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.5))

plt.suptitle('Bell State: Perfect Correlation in BOTH Bases!', 
             fontsize=15, fontweight='bold')
plt.tight_layout()
plt.show()

print("\n🤯 AMAZING: Perfect correlation in BOTH Z and X bases!")
print("   This is the signature of quantum entanglement!")
print("\n💡 Classical systems CANNOT reproduce this!")

## 🎯 CHECKPOINT 2: Can you...

- [ ] Explain how to measure in X basis (H before measurement)?
- [ ] Predict that |Φ+> has 100% correlation in Z basis?
- [ ] Predict that |Φ+> has 100% correlation in X basis?
- [ ] Understand this is different from classical?

Great! Now let's prove this rigorously with Bell's theorem! 🚀

---

## 💡 CONCEPT: Bell's Theorem - The Mathematical Proof

We've seen quantum entanglement gives perfect correlations in multiple bases.

**Bell's Question (1964)**: Can ANY classical (local hidden variable) theory reproduce these correlations?

**Answer**: NO! And we can prove it mathematically! 🎯

## Section 6: The CHSH Inequality - Classical Limit

### 🎲 The CHSH Game Setup

**Players**: Alice and Bob (separated, can't communicate)

**Measurements**:
- Alice can choose angle **θ_A** from {0°, 45°}
- Bob can choose angle **θ_B** from {22.5°, 67.5°}
- Each measurement gives result **+1** or **-1**

**Correlation**: C(θ_A, θ_B) = average of (Alice_result × Bob_result)

### 📐 The CHSH Parameter

We define:
$$S = |C(0°, 22.5°) + C(0°, 67.5°) + C(45°, 22.5°) - C(45°, 67.5°)|$$

### 🔒 Classical Bound (Local Hidden Variables)

**Bell proved**: For ANY classical (local realistic) theory:
$$S_{classical} \leq 2$$

This is the **CHSH inequality**.

### ⚛️ Quantum Prediction

For entangled qubits:
$$S_{quantum} = 2\sqrt{2} \approx 2.828$$

**This VIOLATES the classical bound!** 🤯

### 🎯 What This Means

If we measure S > 2 experimentally:
- ❌ Classical (local hidden variable) theories are RULED OUT
- ✅ Nature is fundamentally quantum!
- 🌌 "Spooky action at a distance" is REAL (but can't send information)

## 💻 IMPLEMENTATION: CHSH Test on Quantum Computer

In [ ]:
# Helper function to measure correlation at specific angles
def measure_correlation(theta_a, theta_b, shots=1000):
    """
    Measure correlation C(θ_A, θ_B) for Bell state.
    
    Args:
        theta_a: Alice's measurement angle (radians)
        theta_b: Bob's measurement angle (radians)
        shots: Number of measurements
        
    Returns:
        Correlation value C = <A × B>
    """
    qc = QuantumCircuit(2, 2)
    
    # Create Bell state |Φ+>
    qc.h(0)
    qc.cx(0, 1)
    
    # Alice's measurement basis rotation
    qc.ry(-2 * theta_a, 0)  # Rotate measurement basis by θ_A
    
    # Bob's measurement basis rotation  
    qc.ry(-2 * theta_b, 1)  # Rotate measurement basis by θ_B
    
    # Measure
    qc.measure([0, 1], [0, 1])
    
    # Execute
    counts = simulator.run(qc, shots=shots).result().get_counts()
    
    # Calculate correlation C = <A × B>
    # Map 0→+1, 1→-1 for each qubit
    correlation = 0
    for outcome, count in counts.items():
        a_result = 1 if outcome[1] == '0' else -1  # Alice (bit 1)
        b_result = 1 if outcome[0] == '0' else -1  # Bob (bit 0)
        correlation += (a_result * b_result) * count / shots
    
    return correlation

print("✅ CHSH correlation function defined!")
print("   Ready to measure quantum correlations at different angles")

In [ ]:
# Perform CHSH experiment
print("🧪 Running CHSH Experiment...\n")

# Define angles (in radians)
# Optimal angles for maximal CHSH violation
theta_a1 = 0              # 0°
theta_a2 = np.pi / 4      # 45°
theta_b1 = np.pi / 8      # 22.5°
theta_b2 = 3 * np.pi / 8  # 67.5°

shots = 5000  # More shots for better statistics

# Measure all 4 correlations
print("Measuring correlations (this may take a moment...)\n")

C1 = measure_correlation(theta_a1, theta_b1, shots)  # C(0°, 22.5°)
print(f"C(0°, 22.5°) = {C1:.4f}")

C2 = measure_correlation(theta_a1, theta_b2, shots)  # C(0°, 67.5°)
print(f"C(0°, 67.5°) = {C2:.4f}")

C3 = measure_correlation(theta_a2, theta_b1, shots)  # C(45°, 22.5°)
print(f"C(45°, 22.5°) = {C3:.4f}")

C4 = measure_correlation(theta_a2, theta_b2, shots)  # C(45°, 67.5°)
print(f"C(45°, 67.5°) = {C4:.4f}")

# Calculate CHSH parameter S
S = abs(C1 + C2 + C3 - C4)

print(f"\n" + "="*50)
print(f"🎯 CHSH PARAMETER: S = {S:.4f}")
print(f"="*50)

# Classical bound
classical_bound = 2.0
quantum_prediction = 2 * np.sqrt(2)

print(f"\n📊 Comparison:")
print(f"   • Classical limit (local hidden variables): S ≤ {classical_bound:.4f}")
print(f"   • Quantum prediction (entanglement): S = {quantum_prediction:.4f}")
print(f"   • Our measurement: S = {S:.4f}")

if S > classical_bound:
    violation = ((S - classical_bound) / classical_bound) * 100
    print(f"\n🎉 BELL INEQUALITY VIOLATED!")
    print(f"   Violation: {violation:.1f}% above classical limit")
    print(f"\n✅ This proves:")
    print(f"   • Nature is NOT classical (local realistic)")
    print(f"   • Quantum entanglement is REAL")
    print(f"   • Einstein was WRONG about 'spooky action'! 🤯")
else:
    print(f"\n⚠️ No violation detected (needs more shots or check circuit)")

# Visualization
fig, ax = plt.subplots(figsize=(10, 6), dpi=150)

categories = ['Classical\nBound', 'Quantum\nPrediction', 'Our\nMeasurement']
values = [classical_bound, quantum_prediction, S]
colors_bar = [COLORS['classical'], COLORS['quantum'], COLORS['accent']]

bars = ax.bar(categories, values, color=colors_bar, 
              edgecolor='black', linewidth=2, alpha=0.8)

# Add horizontal line at classical bound
ax.axhline(y=classical_bound, color='red', linestyle='--', 
           linewidth=2, label='Classical Limit (S = 2)')

ax.set_ylabel('CHSH Parameter S', fontsize=12)
ax.set_title('Bell Test Results: Quantum Violation of Classical Bound', 
             fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(axis='y', alpha=0.3)

# Annotate values on bars
for bar, value in zip(bars, values):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{value:.3f}',
            ha='center', va='bottom', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n🌌 This is one of the most profound results in physics!")
print("   Entanglement is not just correlation - it's fundamentally quantum.")

## 🎯 CHECKPOINT 3: Can you...

- [ ] Explain the CHSH inequality?
- [ ] Understand that S_classical ≤ 2?
- [ ] Understand that S_quantum = 2√2 ≈ 2.828?
- [ ] Explain why S > 2 rules out classical theories?

Excellent! You've proven quantum entanglement is real! 🎉

---

## 🎯 Quick Quiz

Test your understanding of entanglement!

**1. Classical perfect correlation can be explained by:**
- [x] Hidden variables (predetermined values)
- [ ] Quantum superposition
- [ ] Faster-than-light communication

**2. To measure in X basis, we:**
- [ ] Apply X gate before measurement
- [x] Apply H gate before measurement
- [ ] Just measure normally

**3. Bell state |Φ+> has perfect correlation in:**
- [ ] Only Z basis
- [ ] Only X basis
- [x] Both Z and X bases

**4. The classical bound for CHSH is:**
- [ ] S ≤ 1
- [x] S ≤ 2
- [ ] S ≤ 3

**5. Quantum systems can achieve:**
- [ ] S = 2
- [x] S = 2√2 ≈ 2.828
- [ ] S = 4

**6. A CHSH violation (S > 2) proves:**
- [ ] Quantum computers are faster
- [x] Nature is not classical (local realistic)
- [ ] We can communicate faster than light

**7. Separable states like |00>:**
- [x] Respect classical bound (S ≤ 2)
- [ ] Violate Bell's inequality
- [ ] Are maximally entangled

## 🎯 FINAL CHECKPOINT: Can you...

Before moving to the next notebook, verify you can:

- [ ] Explain classical correlation with hidden variables?
- [ ] Measure qubits in different bases (Z, X)?
- [ ] Understand Bell state has correlations in multiple bases?
- [ ] Explain the CHSH inequality and classical bound?
- [ ] Understand why S > 2 rules out classical theories?
- [ ] Run a CHSH test on quantum computer?

If yes to all, congratulations! You've proven quantum entanglement is real! 🎉

---

## 🎓 Notebook 4 Summary

**What you learned:**
✅ Classical correlations use hidden variables (predetermined)  
✅ Quantum correlations from entanglement (no predetermined values)  
✅ Measuring in different bases (Z, X) with basis rotations  
✅ Bell's theorem and CHSH inequality  
✅ Classical bound: S ≤ 2, Quantum: S = 2√2 ≈ 2.828  
✅ Experimental violation proves nature is quantum!  

**Progress**: ⬛⬛⬛⬛⬛⬜⬜ (5/7 completed)

---

## 🚀 Next Step: Notebook 5 - Deutsch's Algorithm

We've proven entanglement is real and fundamentally quantum.

Now let's use quantum properties for **computation**!

**Coming up**:
- The concept of quantum oracle
- Deutsch's algorithm: solve with 1 query (classical needs 2!)
- First concrete example of quantum advantage
- Understanding quantum parallelism

We'll see how quantum computers can actually compute better! 🚀

See you soon! 🎉